## Part 1-1: Greedy with non-optimistic values (Incremental implementation of simple average method)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random 
import itertools

In [ ]:
def rewards_per_step_greedy(steps, n_bandit, n_lever, testbed, global_reward_list, global_optimal_action_list):
    action_count = np.zeros((n_bandit, n_lever))  
    reward_estimates = np.zeros((n_bandit, n_lever))  # initial reward estimates set to zero
    avg_rewards_per_step = []
    optimal_action_count = np.zeros(steps)

    for step in range(steps):
        reward_sum_over_all_bandits_per_step = 0
        optimal_action_chosen_count = 0

        for problem_index in range(n_bandit):  
            maxval = np.amax(reward_estimates[problem_index]) # find the maximum value of the reward for that problem.
            maxval_indices = np.ravel(np.where(reward_estimates[problem_index] == maxval)) #gets the index of that reward in the problem
            # print("maxval and maxval_indices", maxval, maxval_indices)
            random_choice = np.random.choice(maxval_indices)  #in situatuin where there are multiple rewards with the same value
            #: Randomly selects one index from the list of indices with the highest estimated reward. This breaks the tie randomly among the levers that have the same highest estimated reward.
            # print("random choice is" , random_choice)

            # Generate reward from the testbed
            Rn = np.random.normal(testbed[problem_index][random_choice], 1)
            action_count[problem_index][random_choice] += 1
            n = action_count[problem_index][random_choice]
            
            # To check if the action is never taken 
            if n==0:
                reward_estimates[problem_index][random_choice] = 0
            
            else:
                
                # Update reward estimate using the incremental implementation of the simple average method
                reward_estimates[problem_index][random_choice] += (Rn - reward_estimates[problem_index][random_choice]) / n
            
            reward_sum_over_all_bandits_per_step += Rn
            
            # Check if the optimal action was chosen
            optimal_action = np.argmax(testbed[problem_index])
            if random_choice == optimal_action:
                optimal_action_chosen_count += 1

        avg_rewards_per_step.append(reward_sum_over_all_bandits_per_step / n_bandit)
        optimal_action_count[step] = optimal_action_chosen_count / n_bandit

    global_reward_list.append(avg_rewards_per_step)
    global_optimal_action_list.append(optimal_action_count)
    return

# Parameters
steps = 100
n_bandit = 100
n_lever = 10

# Testbed: 1000 sets of ten mean parameters
testbed = np.random.normal(0, 1, (n_bandit, n_lever))

global_reward_list = []
global_optimal_action_list = []

rewards_per_step_greedy(steps, n_bandit, n_lever, testbed, global_reward_list, global_optimal_action_list)

# Average reward at each time step
average_reward_across_runs = np.mean(global_reward_list, axis=0)

# Percentage of time the optimal action is taken
optimal_action_percentage = np.mean(global_optimal_action_list, axis=0) * 100

# Plotting
plt.figure(figsize=(12, 5))

# Plot average reward
plt.subplot(1, 2, 1)
plt.plot(average_reward_across_runs, label='Average Reward')
plt.xlabel('Steps')
plt.ylabel('Average Reward')
plt.title('Average Reward vs. Steps')
plt.legend()

# Plot percentage of optimal action
plt.subplot(1, 2, 2)
plt.plot(optimal_action_percentage, label='% Optimal Action')
plt.xlabel('Steps')
plt.ylabel('% Optimal Action')
plt.title('% Optimal Action vs. Steps')
plt.legend()

plt.tight_layout()
plt.show()

## Part 1-2: Epsilon Greedy with different choices of epsilon 

In [ ]:
def rewards_per_step_greedy(steps, n_bandit, n_lever, testbed, epsilon):
    action_count = np.zeros((n_bandit, n_lever))  
    reward_estimates = np.zeros((n_bandit, n_lever))  # initial reward estimates set to zero
    avg_rewards_per_step = []
    optimal_action_count = np.zeros(steps)

    for step in range(steps):
        reward_sum_over_all_bandits_per_step = 0
        optimal_action_chosen_count = 0

        for problem_index in range(n_bandit):  
            var_random = random.random()
            if (var_random > epsilon):
                maxval = np.amax(reward_estimates[problem_index]) # find the maximum value of the reward for that problem.
                maxval_indices = np.ravel(np.where(reward_estimates[problem_index] == maxval)) #gets the index of that reward in the problem
                random_choice = np.random.choice(maxval_indices)  #in situatuin where there are multiple rewards with the same value
                # Randomly selects one index from the list of indices with the highest estimated reward.
            else:
                random_choice = np.random.randint(n_lever)

            # Generate reward from the testbed
            Rn = np.random.normal(testbed[problem_index][random_choice], 1)
            action_count[problem_index][random_choice] += 1
            n = action_count[problem_index][random_choice]
            
            # To check if the action is never taken 
            if n==0:
                reward_estimates[problem_index][random_choice] = 0
            
            else:
                
                # Update reward estimate using the incremental implementation of the simple average method
                reward_estimates[problem_index][random_choice] += (Rn - reward_estimates[problem_index][random_choice]) / n

            reward_sum_over_all_bandits_per_step += Rn

            # Check if the optimal action was chosen
            optimal_action = np.argmax(testbed[problem_index])
            if random_choice == optimal_action:
                optimal_action_chosen_count += 1

        avg_rewards_per_step.append(reward_sum_over_all_bandits_per_step / n_bandit)
        optimal_action_count[step] = optimal_action_chosen_count / n_bandit

    return avg_rewards_per_step, optimal_action_count

# Parameters
steps = 1000
n_bandit = 1000
n_lever = 10
epsilon_values = [0.0, 0.01, 0.1, 0.2, 0.5]  # 5 different epsilon values
colors = ['r', 'g', 'b', 'k', 'y']  

# Testbed: 1000 sets of ten mean parameters
testbed = np.random.normal(0, 1, (n_bandit, n_lever))

# To store results
all_avg_rewards = []
all_optimal_action_percentages = []

for epsilon in epsilon_values:
    avg_rewards_per_step, optimal_action_count = rewards_per_step_greedy(steps, n_bandit, n_lever, testbed, epsilon)
    all_avg_rewards.append(avg_rewards_per_step)
    all_optimal_action_percentages.append(optimal_action_count * 100)

# Plotting
plt.figure(figsize=(12, 10))

# Plot average reward
plt.subplot(2, 1, 1)
for i, epsilon in enumerate(epsilon_values):
    plt.plot(all_avg_rewards[i], label=f'ε = {epsilon}', color=colors[i])
plt.xlabel('Steps')
plt.ylabel('Average Reward')
plt.title('Average Reward vs. Steps')
plt.legend()

# Plot percentage of optimal action
plt.subplot(2, 1, 2)
for i, epsilon in enumerate(epsilon_values):
    plt.plot(all_optimal_action_percentages[i], label=f'ε = {epsilon}', color=colors[i])
plt.xlabel('Steps')
plt.ylabel('% Optimal Action')
plt.title('% Optimal Action vs. Steps')
plt.legend()

plt.tight_layout()
plt.show()

## Part 1-3: Optimistic Starting values with a greedy approach

In [ ]:
def rewards_per_step_optimistic_greedy(steps, n_bandit, n_lever, testbed, epsilon):
    action_count = np.zeros((n_bandit, n_lever))  
    reward_estimates = np.ones((n_bandit, n_lever)) * 5  # initial reward estimates set to five 
    avg_rewards_per_step = []
    optimal_action_count = np.zeros(steps)

    for step in range(steps):
        reward_sum_over_all_bandits_per_step = 0
        optimal_action_chosen_count = 0

        for problem_index in range(n_bandit):  
            var_random = random.random()
            if (var_random > epsilon):
                maxval = np.amax(reward_estimates[problem_index]) # find the maximum value of the reward for that problem.
                maxval_indices = np.ravel(np.where(reward_estimates[problem_index] == maxval)) #gets the index of that reward in the problem
                random_choice = np.random.choice(maxval_indices)  #in situatuin where there are multiple rewards with the same value
                # Randomly selects one index from the list of indices with the highest estimated reward.
            else:
                random_choice = np.random.randint(n_lever)

            # Generate reward from the testbed
            Rn = np.random.normal(testbed[problem_index][random_choice], 1)
            action_count[problem_index][random_choice] += 1
            n = action_count[problem_index][random_choice]

            # To check if the action is never taken 
            if n==0:
                reward_estimates[problem_index][random_choice] = 0
            
            else:
                # Update reward estimate using the incremental implementation of the simple average method
                reward_estimates[problem_index][random_choice] += (Rn - reward_estimates[problem_index][random_choice]) / n

            reward_sum_over_all_bandits_per_step += Rn

            # Check if the optimal action was chosen
            optimal_action = np.argmax(testbed[problem_index])
            if random_choice == optimal_action:
                optimal_action_chosen_count += 1

        avg_rewards_per_step.append(reward_sum_over_all_bandits_per_step / n_bandit)
        optimal_action_count[step] = optimal_action_chosen_count / n_bandit

    return avg_rewards_per_step, optimal_action_count

# Parameters
steps = 1000
n_bandit = 1000
n_lever = 10
epsilon_values = [0.0, 0.01, 0.1, 0.2, 0.5]  # 5 different epsilon values
colors = ['r', 'g', 'b', 'k', 'y']  

# Testbed: 1000 sets of ten mean parameters
testbed = np.random.normal(0, 1, (n_bandit, n_lever))

# To store results
all_avg_rewards = []
all_optimal_action_percentages = []

for epsilon in epsilon_values:
    avg_rewards_per_step, optimal_action_count = rewards_per_step_optimistic_greedy(steps, n_bandit, n_lever, testbed, epsilon)
    all_avg_rewards.append(avg_rewards_per_step)
    all_optimal_action_percentages.append(optimal_action_count * 100)

# Plotting
plt.figure(figsize=(12, 10))

# Plot average reward
plt.subplot(2, 1, 1)
for i, epsilon in enumerate(epsilon_values):
    plt.plot(all_avg_rewards[i], label=f'ε = {epsilon}', color=colors[i])
plt.xlabel('Steps')
plt.ylabel('Average Reward')
plt.title('Average Reward vs. Steps')
plt.legend()

# Plot percentage of optimal action
plt.subplot(2, 1, 2)
for i, epsilon in enumerate(epsilon_values):
    plt.plot(all_optimal_action_percentages[i], label=f'ε = {epsilon}', color=colors[i])
plt.xlabel('Steps')
plt.ylabel('% Optimal Action')
plt.title('% Optimal Action vs. Steps')
plt.legend()

plt.tight_layout()
plt.show()

## Part 1-4: Gradient Bandit Algorithm (Different Learning rates)

In [ ]:
# Function for softmax distribution
def softmax_distribution(preference_estimates):
    maxval = np.amax(preference_estimates)
    exps = np.exp(preference_estimates - maxval)  # Using property of softmax function
    probabilities = exps / np.sum(exps, axis=0)
    return probabilities

# Gradient Bandit Algorithm
def gradient_bandit(steps, n_bandit, n_lever, step_size, is_baseline_applied,
                    testbed, global_average_reward_list, optimal_choice_list):
    
    action_count = np.zeros((n_bandit, n_lever))  
    Ravg = np.zeros((n_bandit, n_lever))  # avg_reward_estimates
    Hpref = np.zeros((n_bandit, n_lever))  # preference_estimates
    pr_action_t = np.zeros((n_bandit, n_lever))  # probability of action a at time t

    optimal_choice_per_step = []
    mean_reward = 0
    rewards = np.zeros((steps, n_bandit))
    for step in range(steps):
        sum_of_optimal_choice = 0 
        avg_reward = []
        # Loop through all problems 
        for b in range(n_bandit):  
            # calculating the probability of actions based on their preference 
            pr_action_t[b] = softmax_distribution(Hpref[b])
            # chooisesq a random action from n_lever(possible action) according to their probabilities 
            A = np.random.choice(np.arange(n_lever), p=pr_action_t[b])

            # np.arg max will return the index of the maximum value of the problem bandit from testbed and compare it with A (action NOT reward)
            if A == np.argmax(testbed[b]):
                # Add 1 if the condition is true 
                sum_of_optimal_choice += 1 
            # Calculate the reward based on testbed[problem][action]
            Rn = np.random.normal(testbed[b][A], 1)
            rewards[step,b] = Rn
            
            if is_baseline_applied:
                n = step + 1
                mean_reward = (Rn + (n - 1) * mean_reward) / n
            
            Hpref[b][:A] -= step_size * (Rn - mean_reward) * pr_action_t[b][:A] #updates for action 0 to A-1
            # if the value of the current reward is higher than the mean value, then the preference estimates for the actions before them will DECREASRE and otherwise it will increase.
            Hpref[b][A + 1:] -= step_size * (Rn - mean_reward) * pr_action_t[b][A + 1:] #updates actions for A+1 onwards
            #same as above.
            Hpref[b][A] += step_size * (Rn - mean_reward) * (1 - pr_action_t[b][A]) #updates action A only                       

        #Calculates the percentage of hoosing optimal action in each step for every bandit problem
        optimal_choice_per_step.append((sum_of_optimal_choice / n_bandit) * 100)
    # Appendsall of the optimal percentage choices for all steps to global_reward_list
    optimal_choice_list.append(optimal_choice_per_step) # returned for the second plot

    average_rewards = np.mean(rewards,axis = 1)
    global_average_reward_list.append(average_rewards) # returned for the first plot
    return 

# settings
steps = 1000
n_bandit = 1000
n_lever = 10
initial_reward_estimates = np.zeros((n_bandit, n_lever))
testbed = np.random.normal(0, 1, (n_bandit, n_lever))
optimal_choice_list = []
global_average_reward_list = []
baseline_applied = [True, False]  
step_size = [0.1,0.01,0.2]  # n_lever values
combinations = list(itertools.product(step_size, baseline_applied))

for size, baseline in combinations:
    
    print(f"Running gradient_bandit with baseline={baseline} and step_size={size}")
    gradient_bandit(steps, n_bandit, n_lever, size, baseline, testbed, global_average_reward_list, optimal_choice_list)

plt.figure(figsize=(10, 8))


for i, (size,baseline) in enumerate(combinations):
    label = f'(step_size={size}, baseline={baseline})'
    plt.plot(global_average_reward_list[i], label=label)
plt.xlabel('Steps')
plt.ylabel('Average Global Rewards')
plt.legend()
plt.title('Gradient Bandit Algorithm Performance')
plt.show()


for i, (size,baseline) in enumerate(combinations):
    label = f'(step_size={size}, baseline={baseline})'
    plt.plot(optimal_choice_list[i], label=label)
plt.xlabel('Steps')
plt.ylabel('Optimal Action (%)')
plt.legend()
plt.title('Gradient Bandit Algorithm Performance')
plt.show()